### Objectives
1. Get 50 frames from each video
2. At a rate of 1 frame per second of video

In [41]:
# import required packages
import cv2
import os
import moviepy.editor as mpe
from tqdm import tqdm
import pandas as pd
from collections import namedtuple
import numpy as np
from signalwire.rest import Client as signalwire_client
import os


## Capture images from video stream 

The following snippet of code nneds to be called only if the predictions are fed as a video and the frames (pictures) need to be extracted from it.  If the predictions with bounding boxes are captured as a video stream, the images / frames are saved for further processing, which primarily includes checking PPE compliance. 


### 1. Extract & Save Frames from Video

As a first step, read in the file and extract the frames and save them for further processing.

In [4]:
vidcap = cv2.VideoCapture('prediction_animation.mp4')
success,image = vidcap.read()

In [5]:
print(success)
print(image.shape)

True
(700, 1236, 3)


In [6]:
IMG_SAVE_PATH='images/' # folder to save images in

In [5]:
count = 0
success = True
while success:
    success,image = vidcap.read()
    cv2.imwrite(IMG_SAVE_PATH+"frame%d.jpg" % count, image)     # save frame as JPEG file
    if cv2.waitKey(0) == 27:                     # exit if Escape is hit
        break    
    count += 1

### 2. Process video read frame/images

Read the video file from the path and save the images to the specific folder.

In [6]:
VID_PATH_GET='input_video/'
VID_PATH_SAVE='video_images/' # folder to save images in

In [7]:
# get all .mp4 files for processing
lst=os.listdir(VID_PATH_GET) # folder with .mp4 files 
vidlst=sorted([i for i in lst if '.mp4' in i])
print(vidlst)

['prediction_animation.mp4']


### 3. Clean-up folder to hold images

In [8]:
# make/clear frame save dir
try:
    os.mkdir(VID_PATH_SAVE)
except FileExistsError as f:
    print(f)
    pass
    
# clear out save directory for frames
inpt=input('Do you want to delete all .jpeg from directory? y/n ')
if inpt=='y': [os.remove(VID_PATH_SAVE+i) for i in os.listdir('video_images') if '.jpeg' in i]
    #[os.remove(i) for i in os.listdir('video_images//') ]

[WinError 183] Cannot create a file when that file already exists: 'video_images/'
Do you want to delete all .jpeg from directory? y/n y


In [9]:
# get a 50 sec clip with one frame per second with some buffer to start (e.g 20 secs), if the vid is long enough
max_segment_length=50; start=10

err_lst,vid_proc=[],[] # list of vids not processed, list of processed vids

counter=0
for vid in tqdm(vidlst):
    vid = vidlst[0]
    video = mpe.VideoFileClip(VID_PATH_GET+vid)
    start=1 if video.duration < max_segment_length else video.duration-max_segment_length
    finish=max_segment_length+start if video.duration > max_segment_length+start else video.duration

    for t in range(0,int(video.duration),max(1,int(video.duration/50))):
        try:
            video.save_frame(VID_PATH_SAVE+'{}_{}.jpeg'.
                             format(vid.split('.mp4')[0].strip(), counter), t=t) # save frame at t=2 as JPEG
            vid_proc.append((vid,': ',video.duration))
        except:
            err_lst.append('{} had OS error and was not processed'.format(vid))
            pass

        counter+=1

100%|████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:12<00:00, 12.85s/it]


## Process 'Results' file to get objects with valid overlaps

We are primarily looking to see if the person in the frame (assuming, they are at the construction site) have the safety vest and hard hat on them and this can be validated from the bounding box information of the objects.  Since safety vest and hardhats are worn by the person, we can make the assumption that the coordinates of the bounding box will have high overlap or intersection area (compared with the smaller object). 

The logic below takes this approach and identifies if the person in frame has a matching safety vest and/or hardhat identified.  There may be scenarios, where in the object in the frame is cut-off (because of the way the picture was taken) and so, the person may be identified, but the hard hat may not show up in the frame and so, the person will not get tagged as having a hard hat on them. 

In [ ]:
# set path to the fi
VID_PATH_GET='input_video/'
VID_PATH_SAVE='video_images/' # folder to save images in

In [7]:
# define a named tuple to hold objects that need to be compared by their bounding box information
Detection = namedtuple("Detection", ["Box1Category", "Box2Category", "bb_box1", "bb_box2"])

In [8]:
def get_BoundingBox_IOU_Overlap(boxA, boxB):
    # determine the (x, y)-coordinates of the intersection rectangle
    xA = max(boxA[0], boxB[0])
    yA = max(boxA[1], boxB[1])
    xB = min(boxA[2], boxB[2])
    yB = min(boxA[3], boxB[3])
 
    # compute the area of intersection rectangle
    interArea = max(0, xB - xA + 1) * max(0, yB - yA + 1)
 
    # compute the area of both the prediction and ground-truth
    # rectangles
    boxAArea = (boxA[2] - boxA[0] + 1) * (boxA[3] - boxA[1] + 1)
    boxBArea = (boxB[2] - boxB[0] + 1) * (boxB[3] - boxB[1] + 1)
 
    # compute the intersection over union by taking the intersection
    # area and dividing it by the sum of prediction + ground-truth
    # areas - the interesection area
    iou = interArea / float(boxAArea + boxBArea - interArea)
    percent_overlap = interArea / boxBArea
    
    #print("boxAArea: {}, boxBArea: {}, inter-area: {}, and iou: {}, and percent overlap: {}".format(boxAArea, boxBArea, interArea, iou, percent_overlap))
 
    # return the intersection over union value
    return round(iou,4), round(percent_overlap,4)

In [9]:
# example of how the data structure would look like
object_detection = [
    Detection("Person", "Vest", [403, 237, 414, 264], [404, 239, 415, 255]),
    Detection("Person", "Vest", [403, 237, 414, 264], [446, 203, 452, 206]),
    Detection("Person", "HardHat", [403, 237, 414, 264], [407, 237, 413, 241]),
    Detection("Person", "Vest", [403, 237, 414, 264], [410, 263, 425, 275])
]

In [10]:
for detection in object_detection:
    iou = get_BoundingBox_IOU_Overlap(detection.bb_box1, detection.bb_box2)
    #print("iou: ", iou)

### Read the results file for processing

The results file holds information for objects detected, the confidence scores and the coordinate details etc.  For each person identified, we are trying to validate that they are having a safety vest and hard hat on them.  For this, we use the area of overlap and the IOU between the two bounding boxes.

In [15]:
# set to the correct folder where the results file are located
wd=os.path.dirname(os.path.abspath('__file__'))
results_folder = '\\result_files'

filenames = []

for file in os.listdir(str(wd)+results_folder):
    #print(file)
    filenames.append(file)

In [17]:
# function defined t fetch the extent of overlap between 
# two objects and their bounding boxes
def get_object_overlap(parent_row, df_obj):
    parent_overlap=[]
    
    parent_obj=[]
    for sub_index, sub_row in df_obj.iterrows():
        parent_obj.append(get_object_details(parent_row['Category'], sub_row['Category'], 
                                             parent_row['xmin'], parent_row['ymin'], parent_row['xmax'], parent_row['ymax'],
                                             sub_row['xmin'], sub_row['ymin'], sub_row['xmax'], sub_row['ymax']))

    iou_overlap = []
    for detection in parent_obj:
        iou, percent_overlap = get_BoundingBox_IOU_Overlap(detection.bb_box1, detection.bb_box2)
        iou_overlap.append([iou, percent_overlap])
    
    #print(iou_overlap)
    max_overlap=[0.0, 0.0]
    
    for result in iou_overlap:
        if result[1] > max_overlap[1]:
            max_overlap=result
    
    parent_overlap.append(max_overlap)
    
    return parent_overlap


In [18]:
def get_object_details(category1, category2, c1_xmin, c1_ymin, c1_xmax, c1_ymax, c2_xmin, c2_ymin, c2_xmax, c2_ymax):
    return (Detection(category1, category2, [c1_xmin, c1_ymin, c1_xmax, c1_ymax], [c2_xmin, c2_ymin, c2_xmax, c2_ymax]))


In [19]:
def has_overlap(row, overlap_threshold=0.35):
    if (row['hardhat_overlap'][0] > 0) and (row['hardhat_overlap'][1] > overlap_threshold):
        return "Good match"
    elif (row['ymin'] < 144):
        return "Object not in frame"
    else:
        return "No match found"


#### Get API key from local file (used for sending text messages)

In [29]:
def getAPIKeyFromFile(filename):
    import imp
    f = open(filename)
    global val
    val = imp.load_source('val', '', f)
    f.close()

# path to file with API-Key
getAPIKeyFromFile('api_key.py')

# the value is stored in val.signalwire_apikey 

In [38]:
def send_alert(message):
    #FOREMAN_PHONE_NUMBER='+15044007414'
    FOREMAN_PHONE_NUMBER='+15126298572'

    #client = signalwire_client("97aa164c-bcd2-441f-aa28-aee2c522d71d", os.environ['SIGNALWIRE_API_KEY'], signalwire_space_url = 'shrinkray.signalwire.com')
    client = signalwire_client("97aa164c-bcd2-441f-aa28-aee2c522d71d", val.signalwire_apikey, signalwire_space_url = 'shrinkray.signalwire.com')

    message = client.messages.create(
                                  from_='+15122290600',
                                  body=message,
                                  to=FOREMAN_PHONE_NUMBER
                              )

#### Define function to proces a file

In [42]:
def process_files(filenames, threshold=0.35):
    # set the counter for tracking number of frames
    frame_count = 0
    # set the counter to track non-compliant numbers
    non_compliant_count = 0
    total_noncompliant = 0
    
    for file in filenames:
        frame_count += 1
        # do processing specific to the file; read the result file
        df = pd.read_csv('result_files/Sequence02_90.txt', sep=' ',header=None, usecols=[0, 1,2,3,4,5])
        df.columns = ['Category', 'ConfidenceScore', 'xmin', 'ymin', 'xmax', 'ymax']
        
        # capture details specific to each category - person, safetyvest and hardhat
        df_person = df[df['Category']=='person'].reset_index(drop=True)
        df_safetyvest = df[df['Category']=='safetyvest'].reset_index(drop=True)
        df_hardhat = df[df['Category']=='hardhat'].reset_index(drop=True)   

        # get object details for person-safetyvest and person-hardhat (to get the overlap info between them)
        person_hardhat_overlap = []
        person_safetyvest_overlap = []

        for p_index, p_row in df_person.iterrows():
            parent_overlap = get_object_overlap(p_row, df_hardhat)
            person_hardhat_overlap.append(parent_overlap[0])


        for p_index, p_row in df_person.iterrows():
            parent_overlap = get_object_overlap(p_row, df_safetyvest)
            person_safetyvest_overlap.append(parent_overlap[0])

        print(person_hardhat_overlap)
        print(person_safetyvest_overlap)

        df_combined = pd.concat([df_person, pd.DataFrame({'hardhat_overlap': person_hardhat_overlap, 'safetyvest_overlap': person_safetyvest_overlap})] ,axis=1)
        print(df_combined)
        
        df_combined['hh_overlap'] = df_combined.apply(has_overlap, axis=1)
        df_combined['sv_overlap'] = df_combined.apply(has_overlap, axis=1)
        
        df_matches = df_combined[~(df_combined['hh_overlap']=='Good match') & ~(df_combined['sv_overlap']=='Good match')]
        if (df_matches.shape[0] > 1):
            non_compliant_count += 1
            total_noncompliant += 1
        else:
            # reset the counter 
            non_compliant_count = 0
            
        if (frame_count == 3):
            # reset the frame count to restart at 0
            frame_count = 0
        else:
            if (non_compliant_count >= 2):
                send_alert('ATTN Safety Monitor suspects someone onsite is not wearing their PPE!')
         
        print("file: ", file, " has been processed")
        print("Total frames read: {} and total non-compliant so far are: {}", format(str(frame_count), str(non_compliant_count)))
            
            #print("More than one person not meeting PPE compliance")

In [43]:
threshold_value = 0.35

process_files(filenames)

[[0.1212, 0.1766], [0.0455, 0.5], [0.0122, 0.0455], [0.0, 0.0], [0.1433, 1.0], [0.0736, 0.9524]]
[[0.6148, 1.0], [0.4329, 1.0], [0.3852, 1.0], [0.3675, 1.0], [0.0372, 1.0], [0.3356, 0.9529]]
  Category  ConfidenceScore  xmin  ymin  xmax  ymax   hardhat_overlap  \
0   person             0.73   466   142   513   202  [0.1212, 0.1766]   
1   person             0.81   524   179   559   255     [0.0455, 0.5]   
2   person             0.92   381   139   463   293  [0.0122, 0.0455]   
3   person             0.95   276   136   360   305        [0.0, 0.0]   
4   person             0.98   418   184   556   415     [0.1433, 1.0]   
5   person             0.99     5   216   182   367  [0.0736, 0.9524]   

  safetyvest_overlap  
0      [0.6148, 1.0]  
1      [0.4329, 1.0]  
2      [0.3852, 1.0]  
3      [0.3675, 1.0]  
4      [0.0372, 1.0]  
5   [0.3356, 0.9529]  
file:  Sequence01_15.txt  has been processed
Total frames read: {} and total non-compliant so far are: {} 1
[[0.1212, 0.1766], [0.0455, 

### Print Counts

The function below helps to compute the extent of overlap say between the bounding boxes for person and safety vest or for person and hardhat
